# content-aggregation-on-OER-objects

In [190]:
import sys
#!{sys.executable} -m pip install pikepdf
import pikepdf
import urllib.request
import pandas as pd


create temp folder if not already created

In [191]:
from pathlib import Path
Path("temp").mkdir(parents=True, exist_ok=True)

Input an OER Adress

example: ```https://bildungsportal.sachsen.de/opal/oer/10SYjW2AWSFYY```

In [192]:
pdf_url = input("What is the URL of the pdf? ")

What is the URL of the pdf? https://bildungsportal.sachsen.de/opal/oer/10SYjW2AWSFYY


download and open the pdf with pikepdf

create a dictionary for the Metadata

In [193]:
unique = pdf_url.removeprefix("https://bildungsportal.sachsen.de/opal/oer/")
pfad = "temp/" + unique + ".pdf"
if (not os.path.exists(pfad)):
    pdf_filename, headers = urllib.request.urlretrieve(pdf_url, pfad)
else:
    pdf_filename = pfad

pdf = pikepdf.Pdf.open(pdf_filename)
docinfo = pdf.docinfo
metadict = {"Identifier":unique}

write all informations from docinfo into metadict (old code)

In [194]:
#for key, value in docinfo.items():  
#    metadict.update({key:str(value)})

 Autoreninformation (Name✔, Herkunft✔)
 
 Datum der Erstellung✔
 
 Titel✔
 
 Seitenzahl✔
 
 Format✔ (Dokument, Präsentation)
 
 Dateigröße gerundet mit passender Größenmaß ✔

In [195]:
from datetime import datetime

try:
    docinfo["/Author"]
except KeyError:
    metadict.update({"Author":""})
else:
    metadict.update({"Author":str(docinfo["/Author"])})

try:
    docinfo["/Publisher"]
except KeyError:
    metadict.update({"Publisher":""})
else:
    metadict.update({"Publisher":str(docinfo["/Publisher"])})

try:
    docinfo["/CreationDate"]
except KeyError:
    metadict.update({"Datum der Erstellung":""})
else:
    creation = str(docinfo["/CreationDate"])
    date = datetime.strptime(creation.replace("'", ""), "D:%Y%m%d%H%M%S%z")
    metadict.update({"Datum der Erstellung":date.strftime("%x")}) #TODO: change to normal dates
    
try:
    docinfo["/Title"]
except KeyError:
    metadict.update({"Titel":""})
else:
    metadict.update({"Titel":str(docinfo["/Title"])})
    
metadict.update({"Seitenzahl":len(pdf.pages)})

if (pdf.pages[0].mediabox[2] - pdf.pages[0].mediabox[0]) > (pdf.pages[0].mediabox[3] - pdf.pages[0].mediabox[1]):
    metadict.update({"Format":"Dokument"})
else:
    metadict.update({"Format":"Präsentation"})
    
size = os.stat(pdf_filename).st_size
if (size > 1000000000):
    size = size / 1000000000
    sizestr =str(round(size, 1)) + " GB"
elif (size > 1000000):                    
    size = size / 1000000
    sizestr =str(round(size, 1)) + " MB"
elif (size > 1000):
    size = size / 1000
    sizestr =str(round(size, 1)) + " kB"   
else:
    sizestr =str(round(size, 1)) + " B"
metadict.update({"Dateigröße":str(sizestr)})


In [196]:
frame = pd.DataFrame(metadict, [0])
if (os.path.exists("temp/meta.csv")):   
    oldframe = pd.read_csv("temp/meta.csv")
    newframe = pd.concat([oldframe, frame])
    newframe.to_csv("temp/meta.csv", index=False)
    
else:
    frame.to_csv("temp/meta.csv", index=False)
    


In [197]:
import os
pdf.close()
#os.remove(pdf_filename)